In [1]:
!pip install ortools

In [0]:
import numpy as np
from random import randint
from math import ceil
from ortools.sat.python import cp_model

In [0]:
#Initialization
n_students=500
n_courses=20
n_timeslots=4
n_rooms=6
n_days=5

In [0]:
#Importing the csv file as a 2D List

import pandas as pd

df=pd.read_csv("Course_Student_Association_Matrix.csv",header=None)
#Removing the last 2 NAN columns
df = df[df.columns[:-1]]
df = df[df.columns[:-1]]
students_data=df.values
students_data = students_data.tolist()

In [0]:
#Courses taken by each student
courses_for_each_student = [sum(x) for x in zip(*students_data) ]

In [0]:
model = cp_model.CpModel()
courses={}
for i in range(n_students):
  for j in range(n_courses):
    for k in range(n_days):
      for l in range(n_timeslots):
        for m in range(n_rooms):
            courses[(i,j,k,l,m)] = model.NewBoolVar("x(%d,%d,%d,%d,%d)" % (i,j,k,l,m))

In [0]:
#Do Everything 0 if the student is not enrolled in that course

for i in range(n_students):
  for j in range(n_courses):
    for k in range(n_days):
      for l in range(n_timeslots):
        for m in range(n_rooms):
          if students_data[j][i]==0:
            model.Add(courses[(i,j,k,l,m)]==0)

In [0]:
#If a student is enrolled, he/she has to write the exam for that course

for i in range(n_students):
  for j in range(n_courses):
    if students_data[j][i]==1:
      model.Add(sum(courses[i,j,k,l,m] for k in range(n_days) for l in range(n_timeslots) for m in range(n_rooms))==1)

In [0]:
#No 2 courses same time for a student

for i in range(n_students):
  for k in range(n_days):
    for l in range(n_timeslots):
        model.Add(sum(courses[(i,j,k,l,m)] for j in range(n_courses) for m in range(n_rooms))<=1)

In [0]:
#Each room can have 50 or less students which is the room capacity

for j in range(n_courses):
  for k in range(n_days):
    for l in range(n_timeslots):
      for m in range(n_rooms):
        model.Add(sum(courses[i,j,k,l,m] for i in range(n_students))<=50)

In [0]:
#An Exam should not share its room with any other exam

# for k in range(n_days):
#   for l in range(n_timeslots):
#     for m in range(n_rooms):
#       model.Add(sum(courses[i,j,k,l,m] for i in range(n_students) for j in range(n_courses))<=1)

In [0]:
# Same student not more than 2 exams per day

for i in range(n_students):
  for k in range(n_days):
    model.Add(sum(courses[i,j,k,l,m] for j in range(n_courses) for l in range(n_timeslots) for m in range(n_rooms))<=2)

In [0]:
# Exams should not be consecutive

for i in range(n_students):
  for k in range(n_days):
    for l in range(n_timeslots):
      if l<3:
        model.Add(sum((courses[i,j,k,l,m]+courses[i,j,k,l+1,m]) for j in range(n_courses) for m in range(n_rooms))!=2)

In [0]:
#Soft constraint to minimize the number of days
count=[]
for k in range(n_days):
  some_val=sum(courses[i,j,k,l,m] for i in range(n_students) for j in range(n_courses) for l in range(n_timeslots) for m in range(n_rooms))
  if some_val>0:
    count.append(1)
model.Minimize(sum(count))

In [0]:
solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 60.0
status = solver.Solve(model)

In [16]:
result=[]
for i in range(n_students):
  for j in range(n_courses):
    for k in range(n_days):
      for l in range(n_timeslots):
        for m in range(n_rooms):
          if solver.Value(courses[(i,j,k,l,m)])==1:
            print("Student : "+str(i)+" Course : "+str(j)+" Days : "+str(k)+ " Timeslots : "+str(l)+" Room : "+str(m))
            result.append([j+1,k+1,l+1,m+1])

Student : 0 Course : 0 Days : 0 Timeslots : 2 Room : 0
Student : 0 Course : 6 Days : 0 Timeslots : 0 Room : 0
Student : 1 Course : 6 Days : 0 Timeslots : 0 Room : 0
Student : 2 Course : 2 Days : 1 Timeslots : 0 Room : 0
Student : 2 Course : 6 Days : 0 Timeslots : 2 Room : 0
Student : 2 Course : 8 Days : 0 Timeslots : 0 Room : 0
Student : 5 Course : 17 Days : 0 Timeslots : 0 Room : 0
Student : 7 Course : 11 Days : 0 Timeslots : 2 Room : 0
Student : 7 Course : 12 Days : 0 Timeslots : 0 Room : 0
Student : 8 Course : 3 Days : 0 Timeslots : 0 Room : 0
Student : 9 Course : 11 Days : 0 Timeslots : 0 Room : 0
Student : 10 Course : 14 Days : 1 Timeslots : 0 Room : 0
Student : 10 Course : 16 Days : 0 Timeslots : 2 Room : 0
Student : 10 Course : 18 Days : 0 Timeslots : 0 Room : 0
Student : 12 Course : 9 Days : 0 Timeslots : 2 Room : 0
Student : 12 Course : 14 Days : 0 Timeslots : 0 Room : 0
Student : 15 Course : 16 Days : 0 Timeslots : 0 Room : 0
Student : 16 Course : 6 Days : 0 Timeslots : 2 Roo

In [0]:
final_result=[]
for i in result:
  if i not in final_result:
    final_result.append(i)

In [0]:
import numpy
numpy.savetxt('results.csv',final_result,delimiter=",")

In [19]:
no_days=[]
for k in final_result:
  no_days.append(k[1])

print("The number of days required for scheduling the exams are : " + str(len(set(no_days))))

The number of days required for scheduling the exams are : 3
